lstm4

결과요약: RMSE: 23.384016036987305  이상점 제거 함 + normalize 안함-->결과 안 좋음. normalize는 꼭 해야!

lstm4에서는 이상점을 제거하고 normalize를 하지 않는 것을 시도하겠다.

Data import

시도 내용 요약:
주중(1)/주말(2) + 겨울(1)봄(2)여름(3)가을(4) // 

이미 lstm이라는 것이 sequence 개념이 있으므로 시간축(1~397)를 feature로 설정하는 것은 의미가 없을 듯 하여 LSTM 시도2에서는 제외함

https://bigdatascientistblog.wordpress.com/2015/10/02/feature-engineering-with-dates-part-1/

에 따르면 날짜 datestampe에서 9가지의 특성을 끌어낼 수 있다고 한다. 따라서 이를 적용해보고자 한다.

year, month, day, day of week, week of year, hour of day

In [1]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import pandas as pd
tf.set_random_seed(77)


In [3]:
columns=['date','sales']

txs=pd.read_table('./lstmData/lstmPrac2.csv', sep=',',header=None,names=columns )
txs.info()
txs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
date     397 non-null object
sales    397 non-null int64
dtypes: int64(1), object(1)
memory usage: 6.3+ KB


,date,sales
0,2016-01-01,34
1,2016-01-02,41
2,2016-01-03,54
3,2016-01-04,41
4,2016-01-05,35


In [4]:
from datetime import datetime                    

# 1. 기본 feature

In [5]:
year = lambda x: datetime.strptime(x, "%Y-%m-%d" ).year  
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d" ).weekday()
month = lambda x: datetime.strptime(x, "%Y-%m-%d" ).month
# please read docs on how week numbers are calculate
week_number = lambda x: datetime.strptime(x, "%Y-%m-%d" ).strftime('%V')

# 2. 추가 feature

In [6]:
seasons = [0,0,1,1,1,2,2,2,3,3,3,0] #dec - feb is winter, then spring, summer, fall etc
season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d" ).month-1)]

In [7]:
txs['year'] = txs['date'].map(year)
txs['month']=txs['date'].map(month)
txs['week_number']=txs['date'].map(week_number)
txs['season']=txs['date'].map(season)
txs['day_of_week']=txs['date'].map(day_of_week)
# txs['week_number']=txs['date'].map(week_number)
txs

,date,sales,year,month,week_number,season,day_of_week
0,2016-01-01,34,2016,1,53,0,4
1,2016-01-02,41,2016,1,53,0,5
2,2016-01-03,54,2016,1,53,0,6
3,2016-01-04,41,2016,1,01,0,0
4,2016-01-05,35,2016,1,01,0,1
5,2016-01-06,44,2016,1,01,0,2
6,2016-01-07,50,2016,1,01,0,3
7,2016-01-08,42,2016,1,01,0,4
8,2016-01-09,42,2016,1,01,0,5
9,2016-01-10,66,2016,1,01,0,6


pandas를 통해 구한 각 feature는 list()로 우리의 기준type인 list로 변경이 가능

# 3. 추가 가공

## 이상점 제거

normalization, bucketization 을 하여 새로운 열을 생성하는 방향으로

이상점 제거를 위해 평균과 표준편차를 구한다. 이상점의 기준은 일단 평균+-2*sd로 설정

In [8]:
xySales=txs['sales']

In [9]:
xySales=list(xySales)

In [10]:
mean=np.mean(xySales)
mean

35.191435768261968

In [11]:
std=np.std(xySales)
std

36.518152240039889

In [12]:
for i in range(len(xySales)):
    if (xySales[i]<mean-2*std or xySales[i]>mean+2*std):
         xySales[i]=int(mean)
        

In [13]:
xySales

[34,
 41,
 54,
 41,
 35,
 44,
 50,
 42,
 42,
 66,
 50,
 55,
 56,
 53,
 44,
 54,
 54,
 50,
 40,
 49,
 28,
 72,
 71,
 53,
 43,
 38,
 55,
 49,
 43,
 49,
 49,
 44,
 39,
 52,
 45,
 33,
 43,
 40,
 46,
 49,
 50,
 37,
 37,
 45,
 48,
 48,
 38,
 60,
 31,
 35,
 53,
 70,
 62,
 48,
 51,
 49,
 38,
 32,
 39,
 35,
 30,
 36,
 31,
 31,
 44,
 41,
 41,
 45,
 46,
 45,
 41,
 47,
 48,
 40,
 42,
 38,
 38,
 45,
 48,
 62,
 46,
 38,
 62,
 81,
 40,
 45,
 42,
 53,
 53,
 56,
 53,
 47,
 61,
 64,
 62,
 37,
 65,
 54,
 44,
 46,
 50,
 43,
 53,
 63,
 52,
 2,
 0,
 0,
 72,
 59,
 75,
 47,
 44,
 77,
 90,
 93,
 47,
 61,
 77,
 35,
 50,
 58,
 33,
 41,
 36,
 64,
 49,
 53,
 60,
 43,
 41,
 55,
 45,
 18,
 60,
 69,
 54,
 39,
 50,
 44,
 54,
 57,
 82,
 57,
 44,
 56,
 51,
 55,
 28,
 56,
 54,
 36,
 12,
 25,
 41,
 11,
 6,
 9,
 35,
 29,
 24,
 23,
 14,
 21,
 20,
 25,
 23,
 27,
 31,
 16,
 14,
 30,
 32,
 75,
 35,
 26,
 12,
 21,
 23,
 28,
 25,
 31,
 21,
 17,
 10,
 29,
 34,
 28,
 20,
 36,
 23,
 15,
 42,
 28,
 24,
 29,
 20,
 14,
 18,
 27,
 22,


## Normalization

lstm4에서는 생략

## 합쳐서 최종 input XY생성

In [14]:
tempxy=[list(txs['season']),list(txs['day_of_week']),list(txs['week_number']),xySales]

In [15]:
tempxy

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,


In [16]:
xy=np.array(tempxy)

In [17]:
xy

array([['0', '0', '0', ..., '0', '0', '0'],
       ['4', '5', '6', ..., '6', '0', '1'],
       ['53', '53', '53', ..., '04', '05', '05'],
       ['34', '41', '54', ..., '0', '5', '31']], 
      dtype='<U21')

In [18]:
temp=xy.transpose()

In [19]:
temp

array([['0', '4', '53', '34'],
       ['0', '5', '53', '41'],
       ['0', '6', '53', '54'],
       ..., 
       ['0', '6', '04', '0'],
       ['0', '0', '05', '5'],
       ['0', '1', '05', '31']], 
      dtype='<U21')

In [20]:
xy = temp.astype(np.float)

In [21]:
xy

array([[  0.,   4.,  53.,  34.],
       [  0.,   5.,  53.,  41.],
       [  0.,   6.,  53.,  54.],
       ..., 
       [  0.,   6.,   4.,   0.],
       [  0.,   0.,   5.,   5.],
       [  0.,   1.,   5.,  31.]])

In [22]:
#train parameters
seq_length=7
data_dim=4
hidden_dim=10
output_dim=1
learning_rate=0.01
iterations=500
#기본형식
#기본 인자로는 ds-y로 받고 
#처리함수에서 ds를 받아서 'feature 처리모듈'에서 ds에서 요일(주중, 주말), 월(계절) + 날씨와 같은 추가 요인-값을 받는다.


x=xy
y=xy[:,[-1]]

#build a dataset
dataX=[]
dataY=[]
for i in range(0, len(y)-seq_length):
    _x=x[i:i+seq_length]
    _y=y[i+seq_length]
    print(_x,"->",_y)
    dataX.append(_x)
    dataY.append(_y)

# train/test split
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

#input place holders
X=tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y=tf.placeholder(tf.float32, [None, 1])
# build a LSTM network(input: cell shape, type/ )
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# use the last cell's output to calculate Y_pred
Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.tanh) 
# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val))

    # Plot predictions
    plt.plot(testY)
    plt.plot(test_predict)
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()



[[  0.   4.  53.  34.]
 [  0.   5.  53.  41.]
 [  0.   6.  53.  54.]
 [  0.   0.   1.  41.]
 [  0.   1.   1.  35.]
 [  0.   2.   1.  44.]
 [  0.   3.   1.  50.]] -> [ 42.]
[[  0.   5.  53.  41.]
 [  0.   6.  53.  54.]
 [  0.   0.   1.  41.]
 [  0.   1.   1.  35.]
 [  0.   2.   1.  44.]
 [  0.   3.   1.  50.]
 [  0.   4.   1.  42.]] -> [ 42.]
[[  0.   6.  53.  54.]
 [  0.   0.   1.  41.]
 [  0.   1.   1.  35.]
 [  0.   2.   1.  44.]
 [  0.   3.   1.  50.]
 [  0.   4.   1.  42.]
 [  0.   5.   1.  42.]] -> [ 66.]
[[  0.   0.   1.  41.]
 [  0.   1.   1.  35.]
 [  0.   2.   1.  44.]
 [  0.   3.   1.  50.]
 [  0.   4.   1.  42.]
 [  0.   5.   1.  42.]
 [  0.   6.   1.  66.]] -> [ 50.]
[[  0.   1.   1.  35.]
 [  0.   2.   1.  44.]
 [  0.   3.   1.  50.]
 [  0.   4.   1.  42.]
 [  0.   5.   1.  42.]
 [  0.   6.   1.  66.]
 [  0.   0.   2.  50.]] -> [ 55.]
[[  0.   2.   1.  44.]
 [  0.   3.   1.  50.]
 [  0.   4.   1.  42.]
 [  0.   5.   1.  42.]
 [  0.   6.   1.  66.]
 [  0.   0.   2.  50.]
 [

 [  1.   2.  18.  36.]] -> [ 64.]
[[  1.   4.  17.  35.]
 [  1.   5.  17.  50.]
 [  1.   6.  17.  58.]
 [  1.   0.  18.  33.]
 [  1.   1.  18.  41.]
 [  1.   2.  18.  36.]
 [  1.   3.  18.  64.]] -> [ 49.]
[[  1.   5.  17.  50.]
 [  1.   6.  17.  58.]
 [  1.   0.  18.  33.]
 [  1.   1.  18.  41.]
 [  1.   2.  18.  36.]
 [  1.   3.  18.  64.]
 [  1.   4.  18.  49.]] -> [ 53.]
[[  1.   6.  17.  58.]
 [  1.   0.  18.  33.]
 [  1.   1.  18.  41.]
 [  1.   2.  18.  36.]
 [  1.   3.  18.  64.]
 [  1.   4.  18.  49.]
 [  1.   5.  18.  53.]] -> [ 60.]
[[  1.   0.  18.  33.]
 [  1.   1.  18.  41.]
 [  1.   2.  18.  36.]
 [  1.   3.  18.  64.]
 [  1.   4.  18.  49.]
 [  1.   5.  18.  53.]
 [  1.   6.  18.  60.]] -> [ 43.]
[[  1.   1.  18.  41.]
 [  1.   2.  18.  36.]
 [  1.   3.  18.  64.]
 [  1.   4.  18.  49.]
 [  1.   5.  18.  53.]
 [  1.   6.  18.  60.]
 [  1.   0.  19.  43.]] -> [ 41.]
[[  1.   2.  18.  36.]
 [  1.   3.  18.  64.]
 [  1.   4.  18.  49.]
 [  1.   5.  18.  53.]
 [  1.   6.  1

InternalError: Blas GEMM launch failed : a.shape=(273, 14), b.shape=(14, 40), m=273, n=40, k=14
	 [[Node: rnn/while/rnn/basic_lstm_cell/basic_lstm_cell/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](rnn/while/rnn/basic_lstm_cell/basic_lstm_cell/concat, rnn/while/rnn/basic_lstm_cell/basic_lstm_cell/MatMul/Enter)]]
	 [[Node: Sum/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_803_Sum", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'rnn/while/rnn/basic_lstm_cell/basic_lstm_cell/MatMul', defined at:
  File "c:\users\user\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\user\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\use